<a href="https://colab.research.google.com/github/wouterhuls/FlavourPhysicsBND2023/blob/main/sin2beta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

In this exercise we are going to use an LHCb run-2 dataset to measure an angle of the CKM matrix. It is the real dataset, but you do get a little less information than the LHCb analysts have, so the statistical error is a bit larger than in the LHCb publication (https://arxiv.org/abs/2309.09728) . In particular, you will only work with one final state and one 'flavour tagger'. Furthermore, you will miss essential input from a calibration channel that is needed to calibrate the observed asymmetry: for that you can run the 'mixing' workbook. But more about that later.

The learning goals of this exercise are:
* fitting with zfit, a python based modeling package based on Tensorflow. For more instructions, visit the zfit documentation.
* making s-plots and using s-weights for fitting
* extracting the CKM parameter `sin(2*beta)` from a time-dependent asymmetry.

The analysis will consist of two steps. In the first step you will learn about zfit and s-weights. In the second step you will measure sin(2*beta).

# The physics

We will discuss the physics in the lectures, but give a brief summary here. The 'golden channel' for measuring sin(2beta) is the decay $B^0 \to J/\psi K_s$, with $J/\psi\to \mu^+ \mu^-$ and $K_s \to \pi^+ \pi^-$. The final state is a CP-conjugate state: replace every particle by anti-particle and vice versa, and you have the same final state. As a result the final state is equally accessible to a $B^0$ decay and an anti-$B^0$ decay. However, due to the interference of 'mixed' and 'unmixed' decays a difference in the decay time distributions of $B^0$ and anti-$B^0$.

Ignoring $\Delta\Gamma_d$, the decay time distributions can be parametrized as

$ N( B^0 \to  J/\psi K_s )    \propto \frac{e^{-t/\tau}}{\tau} \big[ 1 + S \sin(\Delta m t) - C \cos( \Delta m t) \big] $

$ N( \bar{B}^0 \to J/\psi K_s) \propto \frac{e^{-t/\tau}}{\tau} \big[ 1 - S \sin(\Delta m t) + C \cos( \Delta m t) \big]$

In the SM, $C\approx0$ and $S=\sin(2\beta)$.


To measure the time-dependent oscillations we need three ingredients, namely:
* the decay time
* the flavour of the B at production

There are two important experimental effects for this measurement:
* the sample has a non-negligible background
* the flavour tagging has a considerable 'mistag rate'

In the following we will have to deal with these two effects.

# Prerequites

Install the `zfit` package.

In [ ]:
# @ Prerequisites: install zfit. this will install uproot as well.
!pip install zfit
!pip install hepstats


# Exercise 2

At this location http://www.nikhef.nl/~wouterh/tmp/ksntuple.pkl.bz2 you can find a file with a pandas dataframe with the following fields:
* `mass`: the B candidate invariant mass in MeV
* `decaytime`: the B candidate decaytime in ns
* `qtag`: the charge of the B candidate reconstructed by the flavour tagging algorithm
* `etatag`: the mistagrate assigned by the flavour tagging algorithm

Load the dataset with your favourite tool and draw the reconstructed invariant mass. If you plot it on a log scale, you will find three peaks on a falling exponential background. Knowing that this is a decay to $J/\psi K_s$ and using the PDG, identify the two peaks on the right. What is the left-most peak?

In [ ]:
#@title Example solution
# This is a partial solution to the exercise using uproot. You can also use pyroot if you prefer.
# url ='https://github.com/wouterhuls/sin2beta/raw/main/ntuple_for_BND.root'
#url = 'http://www.nikhef.nl/~wouterh/tmp/ntuple_for_BND.root'
#import uproot
#events = uproot.open(url + ":tree")
#mass = events["mass"].array()

import pandas as pd
df = pd.read_pickle("http://www.nikhef.nl/~wouterh/tmp/ksntuple.pkl.bz2")
mass = df['mass']

import matplotlib.pyplot as plt
plt.hist(mass, bins=200)
plt.yscale('log')
plt.show()

# Hint: you will see a lot more if you plot on a log scale and with more bins!

# Exercise 3

Draw also the B candidate 'decaytime'. The units are in nanoseconds. Compute the average decaytime and its statistical error. How does the answer compare to the average $B^0$ lifetime in the PDG? Give two reasons why the two are different.

Make a profile plot with the average decay time versus the mass.


# Exercise 4

We will now perform a fit to the invariant mass distribution to extract the number of $B^0$ events. Because it may take you too much time to figure this out yourself, we have written most of the code for you.

If you look at the final fit result superimposed on the data set, it looks pretty bad. One reason is the 'signal mass model': it is not very well described by a Gaussian. The other reason is that the $B_s$ peak is not modeled. **Your exercise: add a third component to the mass pdf to model the $B_s$ decay.**

*Hint: The shape of the $B_s$ mass peak is essentially identical to that of the $B_0$. The fit is more stable if the only 'free' parameter of the $B_s$ model is the yield. You can reuse the 'sigma' parameter of the $B_0$ pdf, but the mean is shifted. To get the shifted 'mu' for the $B_s$, use

Bonus exercise:

In [ ]:
import zfit
import numpy as np

# temporary hack, to make sure we can rerun this cell as often as we like.
from collections import OrderedDict
zfit.core.parameter.ZfitParameterMixin._existing_params = OrderedDict()

# Specify the mass range. This is a little tricky: for now we just take all events, but the mass fit needs the right range
massmin = 5160
massmax = 5900

# Create a zfit data set from the dataframe. There is one thing tricky here:
# when the entries are outside the min/max range, they are
# ommitted when reading the dataframe. To prevent that, we make a selection
# beforehand. We also omit the untagged events (q=0), because we do not have
# much use for them later.
in_mass_range = np.logical_and(df['mass']>massmin,df['mass']<massmax)
tagged = df['qtag']!=0
selection = np.logical_and(in_mass_range,tagged)

# Note that this overwrites the original data frame!
df = df[selection]
massobs = zfit.Space("mass",(massmin,massmax))
zdata = zfit.Data.from_pandas( df, obs = massobs )

# create a zfit pdf for the B0 signal
mu_B0 = zfit.Parameter("mu_B0", 5279, 5250, 5300)
sigma_B0 = zfit.Parameter("sigma_B0", 10, 0, 30)
masspdf_B0 = zfit.pdf.Gauss(mu=mu_B0, sigma=sigma_B0, obs=massobs)

# create a zfit pdf for the exponential background
lambd = zfit.Parameter("lambda", -0.001, -1,+1)
masspdf_bkg = zfit.pdf.Exponential(lambd, obs=massobs)

# create a zfit pdf for the Bs signal. here is a trick that you need:
sigma_Bs = sigma_B0
#def mu_Bs_func(params): return params + 87.45 # mass shift
#mu_Bs = zfit.ComposedParameter("mu_Bs",mu_Bs_func, params=[mu_B0])
mu_Bs = zfit.Parameter("mu_Bs", 5366., 5350., 5380.)
masspdf_Bs = zfit.pdf.Gauss(mu=mu_Bs, sigma=sigma_Bs, obs=massobs)

# create an extended PDF from the sum of these
nev = len( mass )
yield_B0  = zfit.Parameter("yield_B0", 0.9*nev, -0.1*nev, 1.1*nev)
yield_Bs  = zfit.Parameter("yield_Bs", 0.05*nev, -0.1*nev, 1.1*nev)
yield_bkg = zfit.Parameter("yield_bkg", 0.1*nev, -0.1*nev, 1.1*nev)
extmasspdf_B0  = masspdf_B0.create_extended(yield_ = yield_B0)
extmasspdf_Bs  = masspdf_Bs.create_extended(yield_ = yield_Bs)
extmasspdf_bkg = masspdf_bkg.create_extended(yield_ = yield_bkg)
pdf_total  = zfit.pdf.SumPDF([extmasspdf_B0, extmasspdf_bkg, extmasspdf_Bs], name="totPDF")

# create a loss function. this is what we will 'minimize'
nll_data = zfit.loss.ExtendedUnbinnedNLL(model=pdf_total, data=zdata)
# create the minimizer. This one uses minuit, but there are various alternatives.
minimizer = zfit.minimize.Minuit()
result = minimizer.minimize(nll_data)
result.hesse()
print(result)

# draw the result
n_bins = 200
npdata = zdata['mass'].numpy()
plot_scaling = len(npdata) / n_bins * massobs.area()
x = np.linspace(massmin,massmax, 1000)
y = pdf_total.pdf(x).numpy()
fig, axes = plt.subplots(2)
axes[1].set_yscale("log")
for i in range(2):
  axis = axes[i]
  color = 'black'
  axis.hist(npdata, color=color, bins=n_bins, histtype="stepfilled", alpha=0.1)
  axis.hist(npdata, color=color, bins=n_bins, histtype="step")
  axis.plot(x, y * plot_scaling, label="Sum - Model", linewidth=2)
  axis.set_xlabel("mass [MeV]")
plt.show()


# Fit with a better mass model

If you look at the final fit result superimposed on the data set, it looks pretty bad. One reason is the 'signal mass model': it is not very well described by a Gaussian. The invariant mass distribution has 'radiative tails' due to QED corrections. Furthermore, the experimental resolution is not very entirely Gaussian.

In LHCb a common solution is to fit with a more complicated model. The most popular solution is the so-called 'double-sided [Crystal Ball](https://en.wikipedia.org/wiki/Crystal_Ball_function)' function. The disadvantage of this model is that it has some highly correlated parameters, which makes the fit a little slow.

Note how much the signal yields change: the error due to the poor mass model is certainly not covered by the statistical error!

In [ ]:
# repeat the fit but with a better mass model
zfit.core.parameter.ZfitParameterMixin._existing_params = OrderedDict()

aL = zfit.Parameter("aL_B0",  1.4, 0.1, 5,floating=True)
aR = zfit.Parameter("aR_B0",  1.4, 0.1, 5,floating=True)
aR = aL
nL = zfit.Parameter("nL_B0", 6, 1., 10, floating=True)
nR = zfit.Parameter("nR_B0", 10, 1., 20,floating=True)

masspdf_B0 = zfit.pdf.DoubleCB(obs=massobs, mu=mu_B0, sigma=sigma_B0, alphal=aL, nl=nL, alphar=aR, nr=nR)
masspdf_Bs = zfit.pdf.DoubleCB(obs=massobs, mu=mu_Bs, sigma=sigma_B0, alphal=aL, nl=nL, alphar=aR, nr=nR)

#masspdf_B0 = zfit.pdf.CrystalBall(obs=massobs, mu=mu_B0, sigma=sigma_B0, alpha=aL, n=nL)
#masspdf_Bs = zfit.pdf.CrystalBall(obs=massobs, mu=mu_Bs, sigma=sigma_B0, alpha=aL, n=nL)


extmasspdf_B0  = masspdf_B0.create_extended(yield_ = yield_B0)
extmasspdf_Bs  = masspdf_Bs.create_extended(yield_ = yield_Bs)
extmasspdf_bkg = masspdf_bkg.create_extended(yield_ = yield_bkg)
pdf_total  = zfit.pdf.SumPDF([extmasspdf_B0, extmasspdf_bkg, extmasspdf_Bs], name="totPDF")

nll_data = zfit.loss.ExtendedUnbinnedNLL(model=pdf_total, data=zdata)
# create the minimizer. This one uses minuit, but there are various alternatives.
result = minimizer.minimize(nll_data)
result.hesse()
print(result)

# draw the result
n_bins = 200
npdata = zdata['mass'].numpy()
plot_scaling = len(npdata) / n_bins * massobs.area()
x = np.linspace(massmin,massmax, 1000)
y = pdf_total.pdf(x).numpy()
fig, axes = plt.subplots(2)
axes[1].set_yscale("log")
for i in range(2):
  axis = axes[i]
  color = 'black'
  axis.hist(npdata, color=color, bins=n_bins, histtype="stepfilled", alpha=0.1)
  axis.hist(npdata, color=color, bins=n_bins, histtype="step")
  axis.plot(x, y * plot_scaling, label="Sum - Model", linewidth=2)
  axis.set_xlabel("mass [MeV]")
plt.show()


# Create the s-weights for background subtraction

To extract the decaytime distribution, we need to subtract the background. We could model the decaytime distribution background and perform a 2D fit to mass and decaytime, but there is an alternative solution that works without a model: Based on the mass fit we can compute so-called s-weights. (See the [sPlot paper](https://arxiv.org/abs/physics/0402083) for details.)  By weighting the events with s-weights, we effectively perform a statistically-optimal background subtraction.

Compute the s-weights using the `hepstats.splot.compute_weights function`. Draw the background subtracted decay time distribution.

In [ ]:
# @ compute s-weights and make a decaytime plot

# use the compute_sweights function from hepstats
from hepstats.splot import compute_sweights
sweights_all = compute_sweights(pdf_total,df['mass'])
sweights_B0 = sweights_all[yield_B0]

# make a background subtracted decay time plot
decaytime = df['decaytime']
plt.hist(decaytime, bins=200, label="all")
plt.hist(decaytime, bins=200,weights = sweights_B0, label="signal")
plt.hist(decaytime, bins=200,weights = 1-sweights_B0, label="background")
plt.yscale("log")
plt.legend()
plt.show()


In [ ]:
# plot the s-weighted tag-weighted decay time distribution
import matplotlib.pyplot as plt
decaytime = df['decaytime']
qtag = df["qtag"]
eta = df["etatag"]
plt.hist(decaytime, bins=200, weights = sweights_B0 * (qtag<0)* (1-2*eta))
plt.hist(decaytime, bins=200, weights = sweights_B0 * (qtag>0)* (1-2*eta))
plt.show()

# suggested binning for asymmetry plot
#tbins = np.concatenate((np.arange(0.0000,5.0,step=0.5),[5.6,6.2,7.0,7.8,8.7,9.7,10.8,12.0,13.4,15.]))
tbins = np.array([0.002,0.5,1.0,1.5,2.0,2.5,3.0,3.5,4.0,4.5,5.2,6.2,7.5,8.8,10.2,11.5,15.])
tbins = tbins/1000.

# choose 20 bins with equal number of events
#tbins = np.quantile(decaytime, np.linspace(start=0.0,stop=1.0,num=21)[1:])
#tbins[-1] = 0.015
#tbins[0]  = 0.0002

# The minus one has something to do with the meaning of the tag
qD = qtag*(1-2*eta)*-1
wqDsum, bin_edges  = np.histogram(decaytime,bins=tbins,weights=sweights_B0*qD)
wqD2sum, bin_edges = np.histogram(decaytime,bins=tbins,weights=sweights_B0*qD*qD)
w2qD2sum, bin_edges = np.histogram(decaytime,bins=tbins,weights=sweights_B0*sweights_B0*qD*qD)
asymmetry    = wqDsum / wqD2sum
asymmetryerr = np.sqrt(w2qD2sum) / wqD2sum

# compute in every bin the average decay time
wtsum, bin_edges = np.histogram(decaytime,bins=tbins,weights=sweights_B0*decaytime)
wsum,  bin_edges = np.histogram(decaytime,bins=tbins,weights=sweights_B0)
avtime = wtsum / wsum

# now draw points with both vertical and horizontal errors
xerrors = [avtime-bin_edges[:-1],bin_edges[1:]-avtime]
plt.errorbar(x=avtime, y=asymmetry, xerr=xerrors, yerr=asymmetryerr,fmt='o')
plt.show()



# Fit the asymmetry

The theoretical model for the mixing asymmetry is

$A_{CP} = S \sin(\Delta m\: t) - C \cos(\Delta m\:t ) $

Fit this model to the data taking into account the dilution.

Comparing your final result to that in the paper, you will notice that it is quite far off. The reason is that the flavour tag dilution is not well calibrated. The calibration factor can be obtained from a control channel. In the 'mixing' workbook the $B_d \to J/\psi K^{*0}$ control channel is used to extract the dilution scale factor.


In [ ]:
# The code below performs the fit to the binned asymmetry.

from tensorflow.python.dlpack.dlpack import from_dlpack
zfit.core.parameter.ZfitParameterMixin._existing_params = OrderedDict()

# declare the parameters. You can also float deltaM if you like.
deltaM = zfit.Parameter("DeltaM",507.,450,550,floating=False)
S = zfit.Parameter("S",0.5,-1.5,1.5,floating=True)
C = zfit.Parameter("C",0.,-1.5,1.5,floating=True)
# define a function that evaluates the chi2 using the asymmetry computed above
def chi2( params ):
  S = params[0]
  C  = params[1]
  dm = params[2]
  cosdmt = np.cos(dm * avtime)
  sindmt = np.sin(dm * avtime)
  res = (S*sindmt - C*cosdmt) - asymmetry
  var = np.square(asymmetryerr)
  chi2 = np.sum( np.square(res)/var)
  return chi2
# create a loss function and minimize it
loss = zfit.loss.SimpleLoss(chi2, [S,C,deltaM], errordef=1)
result = minimizer.minimize(loss)
result.hesse()
print(result)

# draw the result
plt.errorbar(x=avtime, y=asymmetry, xerr=xerrors, yerr=asymmetryerr,fmt='o')
x = np.linspace(0,0.015, 1000)
y = S*np.sin(x*deltaM)-C*np.cos(x*deltaM)
plt.plot(x,y)
plt.show()



# Perform an unbinned maximum likelihood fit

Build a PDF and fit it to the unbinned dataset. How do the errors compare to the binned fit?


In [ ]:
# this solution works but we'll once need to figure out how to do the normalization properly.
import tensorflow as tf
class BMixingPDF(zfit.pdf.BasePDF):
    """ Generic decay PDF with mixing and CP violation"""
    def __init__(self,obs,S,C,deltaM):
        self.S = S
        self.C = C
        self.deltaM = deltaM
        paramdict = { x.name : x for x in [S,C,deltaM]}
        super().__init__(obs=obs, params=paramdict, name="CPDecayPDF")
    def _norm(): return 1
    def _unnormalized_pdf(self, x):
      #tf.print(x)
      decaytime, qD = zfit.z.unstack_x(x)
      return (1 +  qD * (S * tf.sin(self.deltaM*decaytime) - C *tf.cos(self.deltaM*decaytime)))
    def pdf(self,x,norm=False): return self._unnormalized_pdf(x)

fitobs = zfit.Space("decaytime",(0.,0.015)) * zfit.Space("qD",(-2,2))
if "qD" not in df: df["qD"] = qD
zdata = zfit.Data.from_pandas( df, obs=fitobs, weights=sweights_B0 )

pdf  = BMixingPDF(obs=fitobs,S=S,C=C,deltaM=deltaM)
nll_data = zfit.loss.UnbinnedNLL(model=pdf, data=zdata)
minimizer = zfit.minimize.Minuit()
result = minimizer.minimize(nll_data)
result.hesse()
print(result)

# draw the result
plt.errorbar(x=avtime, y=asymmetry, xerr=xerrors, yerr=asymmetryerr,fmt='o')
x = np.linspace(0,0.015, 1000)
y = S*np.sin(x*deltaM)-C*np.cos(x*deltaM)
plt.plot(x,y)
plt.show()


